In [1]:
#!pip install tensorflow==2.14
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#py파일이어야 임포트 됨
from KNN_functions import *
from Customized_CNN import *

In [3]:
def Accuracy(df_2columns): # 0열은 예측값, 1열은 실제값
    print("df_2columns (showing ten rows): \n", df_2columns.head(10))
    
    total = df_2columns.shape[0]
    print("total : ", total)
    correct_count = 0
    
    df_2columns = df_2columns.reset_index(drop=True)
    for i in range(df_2columns.shape[0]):
        if (df_2columns.iloc[i,0] == df_2columns.iloc[i,1]):
            correct_count += 1
            
    print("correct_count : ", correct_count)
    return correct_count / total

In [4]:
def AMTCount(data, isLearn):
    if(not isLearn):
        #print("data\n\n", data)
        #print("df_train_global\n\n", df_train_global)
#         df_train_temp = df_train_global.copy()
#         del df_train_temp["DAY"]
#         print("df_train_temp\n\n", df_train_temp)
        data = pd.concat([df_train_global, data], axis=0)
    amtCounted = pd.DataFrame([], columns=["CC_NUM", "AMT_COUNT"])
    if(isLearn):
        amtCounted["AMT_COUNT"] = ((data["CC_NUM"].to_frame())).value_counts()
        #amtCounted["AMT_COUNT"] = (data.to_frame()).value_counts()
        amtCounted["CC_NUM"] = (data["CC_NUM"].value_counts()).index
        amtCounted["CC_NUM"] = amtCounted["CC_NUM"].astype("int")
        amtCounted.reset_index(drop=True, inplace = True)
        amtCounted = amtCounted.sort_values(by=["CC_NUM"] ,ascending=True)
        #print("@@@@@@@@@@@@@@@@@", amtCounted)
        return amtCounted
    else: #data는 1행임, amtCounted는 빈 df
        amtCounted["AMT_COUNT"] = (data["CC_NUM"]).value_counts()
        #amtCounted["AMT_COUNT"] = (data.to_frame()).value_counts()
        amtCounted["CC_NUM"] = (data["CC_NUM"].value_counts()).index
        amtCounted["CC_NUM"] = amtCounted["CC_NUM"].astype("int64")
        amtCounted.reset_index(drop=True, inplace = True)
        amtCounted = amtCounted.sort_values(by=["CC_NUM"] ,ascending=True)
        return amtCounted

In [5]:
def FraudModel(data, isLearn, alpha, beta, gamma, delta1, delta2, omega, epsilon, mu):
    
    if(not isLearn):
        #data datatype is Series
        data = data.to_frame().T
    amtCount = AMTCount(data, isLearn)
    
    
    if(isLearn):
        KNNforFraud(data, isLearn, beta, delta1, epsilon)
        DifferenceforFraud(data, isLearn, gamma, delta2, omega, mu)
        return 2
    else:
        currentCCNUM = data["CC_NUM"]
        currentCCNUM = currentCCNUM.item()
        amtCount.reset_index(drop=True, inplace=True)
        #print("@#$$%#@$^#%$^#$^\n", amtCount)
        amt_count_temp = np.nan
        for row in amtCount.itertuples():
            #print(row.Index, row.CC_NUM, row.AMT_COUNT)
            if(currentCCNUM == row.CC_NUM):
                amt_count_temp = row.AMT_COUNT
        #print("**amt_count_temp : ", amt_count_temp)
        if(amtCount[amtCount["CC_NUM"] == currentCCNUM].empty or amt_count_temp < alpha):
            #print("Decision value is___", amtCount[amtCount["CC_NUM"] == currentCCNUM])
            fraud_predict = KNNforFraud(data, isLearn, beta, delta1, epsilon)
        else:
            #print("Decision value is___", amtCount[amtCount["CC_NUM"] == currentCCNUM])
            fraud_predict = DifferenceforFraud(data, isLearn, gamma, delta2, omega, mu)
        return fraud_predict

In [6]:
global df_train_x_global
global df_train_y_global
global df_val_x_global
global df_val_y_global


df_train_x_global = pd.read_csv("../data/splitted/Fraud_Detection_train_features.csv")
df_train_y_global = pd.read_csv("../data/splitted/Fraud_Detection_train_target.csv")
df_val_x_global = pd.read_csv("../data/splitted/Fraud_Detection_validation_features.csv")
df_val_y_global = pd.read_csv("../data/splitted/Fraud_Detection_validation_target.csv")

#다루기 쉽게 하기 위해 IS_FRAUD를 int형으로 변경
df_train_y_global = df_train_y_global.astype("int")
df_val_y_global = df_val_y_global.astype("int")

global df_train_global
global df_val_global

df_train_global = pd.concat([df_train_x_global, df_train_y_global], axis=1)
df_val_global = pd.concat([df_val_x_global, df_val_y_global], axis=1)

print(df_train_global["IS_FRAUD"].value_counts())
print("Least frequent CC_NUM count is ", df_val_global["CC_NUM"].value_counts().min())
print("Most frequent CC_NUM count is ", df_val_global["CC_NUM"].value_counts().max())

model_result = FraudModel(data=df_train_global, isLearn=True, alpha=4, beta=7, gamma=4, delta1=0.5, delta2=0.5, omega=3, epsilon=5, mu=5)

0    22223
1     5791
Name: IS_FRAUD, dtype: int64
Least frequent CC_NUM count is  1
Most frequent CC_NUM count is  28
Epoch 1/5
876/876 [==============================] - 1s 568us/step - loss: 0.1802 - accuracy: 0.9715
Epoch 2/5
876/876 [==============================] - 0s 570us/step - loss: 0.0320 - accuracy: 0.9878
Epoch 3/5
876/876 [==============================] - 1s 576us/step - loss: 0.0301 - accuracy: 0.9890
Epoch 4/5
876/876 [==============================] - 1s 588us/step - loss: 0.0297 - accuracy: 0.9893
Epoch 5/5
876/876 [==============================] - 0s 539us/step - loss: 0.0295 - accuracy: 0.9893
ccnum_fraud_row : 1000
ccnum_fraud_row : 2000
ccnum_fraud_row : 3000
ccnum_fraud_row : 4000
ccnum_fraud_row : 5000
ccnum_fraud_row : 6000
ccnum_fraud_row : 7000
ccnum_fraud_row : 8000
ccnum_fraud_row : 9000
ccnum_fraud_row : 10000
ccnum_fraud_row : 11000
ccnum_fraud_row : 12000
ccnum_fraud_row : 13000
ccnum_fraud_row : 14000
ccnum_fraud_row : 15000
ccnum_fraud_row : 16000
c

## 이제 validation을 해 보자
### FraudModel당 하나의 row(데이터)를 넣어야 한다!

In [ ]:
predict_list = []
# for row_idx in range(df_val_x_global.shape[0]):

how_many = df_val_y_global.shape[0] # 전체 검증

for row_idx in range(how_many):
    if((row_idx + 1) % 100 == 0):
        print("%dth data is being validated" %(row_idx + 1))
    model_predicted=FraudModel(data=df_val_global.loc[row_idx], isLearn=False, alpha=4, beta=7, gamma=4, delta1=0.5, delta2=0.5, omega=3, epsilon=5, mu=5)
    predict_list.append(model_predicted)

#print(predict_list)
#print(type(predict_list))


pred_and_real = pd.DataFrame({"PREDICTED":predict_list})
pred_and_real = pred_and_real.astype("int")
pred_and_real = pd.concat([pred_and_real, df_val_y_global.head(how_many)], axis = 1)
print("Accuracy : ", Accuracy(pred_and_real))

100th data is being validated
200th data is being validated
300th data is being validated
400th data is being validated
500th data is being validated
600th data is being validated
700th data is being validated
800th data is being validated
900th data is being validated
1000th data is being validated
1100th data is being validated
1200th data is being validated
1300th data is being validated
1400th data is being validated
1500th data is being validated
1600th data is being validated
1700th data is being validated
1800th data is being validated
1900th data is being validated
2000th data is being validated
2100th data is being validated
2200th data is being validated
2300th data is being validated
2400th data is being validated
2500th data is being validated
1/1 [==============================] - 0s 47ms/step
2600th data is being validated
2700th data is being validated
2800th data is being validated
1/1 [==============================] - 0s 10ms/step
2900th data is being validated
3000th